In [2]:
import librosa
from src import acoustic_util
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import scipy.signal
from orca_hls_utils.DateRangeHLSStream import DateRangeHLSStream
import time
import os

ImportError: dlopen(/Users/grantsavage/.local/share/virtualenvs/capstone-51pnZIVg/lib/python3.9/site-packages/skimage/_shared/geometry.cpython-39-darwin.so, 0x0002): tried: '/Users/grantsavage/.local/share/virtualenvs/capstone-51pnZIVg/lib/python3.9/site-packages/skimage/_shared/geometry.cpython-39-darwin.so' (mach-o file, but is an incompatible architecture (have 'arm64', need 'x86_64')), '/System/Volumes/Preboot/Cryptexes/OS/Users/grantsavage/.local/share/virtualenvs/capstone-51pnZIVg/lib/python3.9/site-packages/skimage/_shared/geometry.cpython-39-darwin.so' (no such file), '/Users/grantsavage/.local/share/virtualenvs/capstone-51pnZIVg/lib/python3.9/site-packages/skimage/_shared/geometry.cpython-39-darwin.so' (mach-o file, but is an incompatible architecture (have 'arm64', need 'x86_64'))
It seems that scikit-image has not been built correctly.

Your install of scikit-image appears to be broken.
Try re-installing the package following the instructions at:
https://scikit-image.org/doc

In [2]:
from src import pipeline
from src import hydrophone

ImportError: dlopen(/Users/grantsavage/.local/share/virtualenvs/capstone-51pnZIVg/lib/python3.9/site-packages/pandas/_libs/interval.cpython-39-darwin.so, 0x0002): tried: '/Users/grantsavage/.local/share/virtualenvs/capstone-51pnZIVg/lib/python3.9/site-packages/pandas/_libs/interval.cpython-39-darwin.so' (mach-o file, but is an incompatible architecture (have 'arm64', need 'x86_64')), '/System/Volumes/Preboot/Cryptexes/OS/Users/grantsavage/.local/share/virtualenvs/capstone-51pnZIVg/lib/python3.9/site-packages/pandas/_libs/interval.cpython-39-darwin.so' (no such file), '/Users/grantsavage/.local/share/virtualenvs/capstone-51pnZIVg/lib/python3.9/site-packages/pandas/_libs/interval.cpython-39-darwin.so' (mach-o file, but is an incompatible architecture (have 'arm64', need 'x86_64'))

In [ ]:
def plot_noise(testdf, name, output_path=None, save = False):
    fig, ax = plt.subplots()
    spec = ax.pcolormesh(testdf.index,testdf.columns,testdf.values.transpose())
    ax.set_yscale('log')
    ax.set_ylim((50, 20000))
    fig.autofmt_xdate(rotation=45)
    fig.gca().xaxis.set_major_formatter(mdates.DateFormatter('%H:%M:%S'))
    #fig.gca().xaxis.set_major_locator(mdates.SecondLocater())
    fig.colorbar(spec,ax=ax, label="dB relative to ancient ambient")
    fig.set_size_inches(10, 10)
    plt.title(name)
    if save:
        fig.savefig(output_path,
                    dpi=80,
                    bbox_inches="tight",
                    pad_inches=0.0)
        
        fig.canvas.draw()
        fig.canvas.flush_events()
        plt.close(fig)
    
    return fig

In [ ]:
orcasound_lab = hydrophone.Hydrophone.ORCASOUND_LAB
ship = pipeline.NoiseAnalysisPipeline(orcasound_lab, delta_f=10, delta_t=1, bands=3)

In [ ]:
start_time = datetime.datetime(2023,2,10,22,00,00)
end_time = datetime.datetime(2023,2,10,23,00,00)

ship_noise_psd, ship_noise_broad = ship.generate_psds(start=start_time, end=end_time)

In [ ]:
aa = acoustic_util.ancient_ambient(ship_noise_broad)

In [ ]:
time_steps = ['0.1', '0.5', '1', '10']
oct_steps = [1, 3, 6, 12, 24]
start_time = datetime.datetime(2023,2,10,22,00,00)
end_time = datetime.datetime(2023,2,10,23,00,00)

for time in range(len(time_steps)):
    for oct in range(len(oct_steps)):
        ship = pipeline.NoiseAnalysisPipeline(orcasound_lab, delta_f=10, delta_t=float(time_steps[time]), bands=int(oct_steps[oct]))
        f_name = str(time_steps[time])+'s_'+str(oct_steps[oct])+'oct'
        out_df, out_rms = ship.generate_psds(start=start_time, end=end_time, ref=aa)
        plot_noise(out_df, f_name, 'filepath{}.png'.format(f_name), save=True)
        out_df.columns = out_df.columns.map(str)
        out_df.to_parquet('filepath{}.parquet.gzip'.format(f_name))

In [ ]:
names = ['0.1s_1oct', '0.1s_3oct', '0.1s_6oct', '0.1s_12oct', '0.1s_24oct',
         '0.5s_1oct', '0.5s_3oct', '0.5s_6oct', '0.5s_12oct', '0.5s_24oct',
         '1s_1oct', '1s_3oct', '1s_6oct', '1s_12oct', '1s_24oct',
         '10s_1oct', '10s_3oct', '10s_6oct', '10s_12oct', '10s_24oct']
images = [plt.imread('filepath'+name+'.png') for name in names]

fig, axes = plt.subplots(4, 5)

i=0
for row in [0, 1, 2, 3]:
    for column in [0, 1, 2, 3, 4]:
        ax = axes[row, column]
        ax.set_title(names[i])
        ax.axis('off')
        ax.imshow(images[i])
        i = i + 1

fig.set_size_inches(10, 10)
plt.show()
fig.savefig('filepath/all.png',
            dpi=80,
            bbox_inches="tight",
            pad_inches=0.0)